In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
from tensorflow import keras

In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
cols = ["sentiment","text"]
train_data = pd.read_csv(
    "./test.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)


In [4]:
train_data.count()

sentiment    400000
text         400000
dtype: int64

In [5]:
data = train_data

In [6]:
def clean_tweet(tweet):
    tweet2 = BeautifulSoup(tweet, "lxml").get_text()
    tweet2 = re.sub(r"@[A-Za-z0-9]+", ' ', tweet2)
    tweet2 = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet2)
    tweet2 = re.sub(r"[^a-zA-Z.!?']", ' ', tweet2)
    tweet2 = re.sub(r" +", ' ', tweet2)
    return tweet2

In [7]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [8]:
print(data_clean[0])

My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this no matter black white young old male female EVERYBODY says one thing Who was that singing ? 


In [9]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [10]:
set(data_labels)

{0, 1}

In [11]:
MAX_LEN=512

In [12]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file("Tokenizer")

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [13]:
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [14]:
Dcnn=keras.models.load_model("./Model/")

2021-11-06 19:03:37.521301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-06 19:03:37.532210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-06 19:03:37.532638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-06 19:03:37.533668: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [15]:
Dcnn.summary()

Model: "dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  33694720  
_________________________________________________________________
conv1d (Conv1D)              multiple                  102500    
_________________________________________________________________
global_max_pooling1d (Global multiple                  0         
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  153700    
_________________________________________________________________
global_max_pooling1d_1 (Glob multiple                  0         
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  204900    
_________________________________________________________________
global_max_pooling1d_2 (Glob multiple                  0      

In [ ]:
#heckpoint_path = "./ckpt"
#kpt = tf.train.Checkpoint(Dcnn=Dcnn)
#kpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
#f ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print("Latest checkpoint restored!")

In [ ]:
results = Dcnn.evaluate(data_inputs,data_labels)
print(results)

2021-11-06 19:03:38.763990: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 819200000 exceeds 10% of free system memory.
2021-11-06 19:03:39.208638: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-06 19:03:39.794491: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


10664/12500 [========================>.....] - ETA: 26s - loss: 0.2758 - accuracy: 0.9135